<a href="https://colab.research.google.com/github/LBY-L/MineColab/blob/master/MineColab_forked_by_LBY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from IPython.core.display import display, HTML
display(HTML('<p><img style="display: block; margin-left: auto; margin-right: auto;" src="https://github.com/LBY-L/MineColab/blob/master/Logo.png?raw=true" alt="" width="170" height="136" /></p><h1 style="text-align: center;"><span style="color: #99ccff;">MineColab</span></h1><hr /><h2 style="text-align: center;"><span style="color: #ccffcc;">Forked By LBY_L</span></h2><h2 style="text-align: center;"><span style="color: #99cc00;">Run Minecraft Server On Google Colab</span></h2>'))

In [ ]:
#@title Accept Eula
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Minecraft-server"
!echo "eula=true" > eula.txt

In [ ]:
#@title Run
#@markdown #### **This will create a folder named "Minecraft-server" in your [Google Drive](https://drive.google.com/) and here put your server jar**
#@markdown #### **Here put the name of your server jar of your server**
JarName = "" #@param {type:"string"}
#@markdown #### **Select if you server use Paper or no**
Paper = "No" #@param ["Yes", "No"]
#@markdown #### **You can select between [Ngrok](https://ngrok.com/) or [PlayIt](https://playit.gg/) if you will use Ngrok you need complete NgrokAuthToken and NgrokRegion**
TunnelService = "ngrok" #@param ["ngrok", "playit"] 
#@markdown #### **Here put your ngrok token, if you haven't it, get it [here](https://dashboard.ngrok.com/get-started/your-authtoken)** 
NgrokAuthToken = "" #@param {type:"string"}
#@markdown #### **Here is the region where is will be hosted your minecraft**

#@markdown * #### `us` - United States (Ohio)

#@markdown * #### `eu` - Europe (Frankfurt)

#@markdown * #### `ap` - Asia/Pacific (Singapore)

#@markdown * #### `au` - Australia (Sydney)

#@markdown * #### `sa` - South America (Sao Paulo)

#@markdown * #### `jp` - Japan (Tokyo)

#@markdown * #### `in` - India (Mumbai)
NgrokRegion = "us" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"]
import os
import re
import json
from IPython.display import clear_output
# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
!mkdir "/content/drive/MyDrive/Minecraft-server"
%cd "/content/drive/MyDrive/Minecraft-server"
clear_output()
!sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."
!sudo apt-get install openjdk-8-jre-headless &>/dev/null && echo "Yay! Openjdk8 has been successfully installed." || echo "Failed to install OpenJdk8."

# Java arguments.
if Paper == "Yes":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx6144M -Xms6144M"


print("Procedding to use", TunnelService)
if TunnelService == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok
  clear_output()
  # Ask for the ngrok authtoken
  !ngrok authtoken $NgrokAuthToken # login to ngrok
  # Sets default ngrok region
  conf.get_default().region = NgrokRegion  # Change this to whichever region you want
  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
  print('Starting server...')
  !java $memory_allocation $server_flags -jar $JarName nogui

elif TunnelService == "playit":
  clear_output()
  !curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
  !sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
  !sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
  print('Starting server...')
  !playit & java $memory_allocation $server_flags -jar $JarName nogui
